# Sphere Reconstruction experiment with patches generating process (diversity condition holds)

g - patching the sphere


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '..'))

import torch
from pathlib import Path

from data.generation import Patches
from visualization_utils.spheres import visualize_spheres_side_by_side, scatter3d_sphere
from encoders import SphericalEncoder

from encoders import get_mlp

import matplotlib.pyplot as plt
from simclr.simclr import SimCLR
from spaces import NSphereSpace

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create figures directory
figures_dir = Path('figures')
figures_dir.mkdir(exist_ok=True)

print('Using', device)

In [ ]:
g_patches = Patches(5, device)

tau = 0.3
kappa = 1 / tau

iterations = 10000
batch = 6144

sphere = NSphereSpace(3)

sample_pair_fixed = lambda batch: sphere.sample_pair_vmf(batch, kappa)
sample_uniform_fixed = lambda batch: sphere.uniform(batch)

f = SphericalEncoder(hidden_dims=[128, 256, 256, 256, 256, 256, 256, 128])

h = lambda z: f(g_patches(z))

z = sphere.uniform(100000)

fig = scatter3d_sphere(plt, z.cpu(), g_patches(z.to(device)).cpu(), s=10, a=.8)
fig.savefig(figures_dir / 'patches_data_generating_process.png', dpi=150, bbox_inches='tight')
plt.show()

fig = scatter3d_sphere(plt, z.cpu(), z.cpu(), s=10, a=.8)
fig.savefig(figures_dir / 'patches_original.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
from visualization_utils.scoring import plot_scores
from experiment_utils.linear import linear_unrotation

f = SphericalEncoder(hidden_dims=[256, 512, 1024, 1024, 1024, 1024, 512, 256])

iterations = 20000

simclr_vmf = SimCLR(
    f, g_patches, sample_pair_fixed, sample_uniform_fixed, tau, device
)

f, scores = simclr_vmf.train(batch, iterations)

h = lambda z: f(g_patches(z))

z = sphere.uniform(1000).to(device)
z_enc = h(z).to(device)

fig_scores = plot_scores(plt, scores)
fig_scores.savefig(figures_dir / 'patches_training_scores.png', dpi=150, bbox_inches='tight')
plt.show()

fig = visualize_spheres_side_by_side(plt, z.cpu(), z_enc.cpu())
fig.savefig(figures_dir / 'patches_after_training_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

z = sphere.uniform(100000).to(device)
z_enc = h(z)

fig = scatter3d_sphere(plt, z.cpu(), z_enc.cpu(), s=10, a=.8)
fig.savefig(figures_dir / 'patches_after_training_encoded.png', dpi=150, bbox_inches='tight')
plt.show()

fig = scatter3d_sphere(plt, z.cpu(), z.cpu(), s=10, a=.8)
fig.savefig(figures_dir / 'patches_after_training_original.png', dpi=150, bbox_inches='tight')
plt.show()

# Unrotation
z_unrotated = linear_unrotation(z, z_enc)
fig = scatter3d_sphere(plt, z.cpu(), z_unrotated, s=10, a=.8)
fig.savefig(figures_dir / 'patches_unrotated.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"All figures saved to {figures_dir}/")

In [11]:
from sklearn import linear_model

lin_reg = linear_model.LinearRegression()

z_h = h(z.to(device)).cpu()

lin_reg.fit(z.detach().cpu().numpy(), z_h.detach().cpu().numpy())

hz_unrotated = lin_reg.predict(z_enc.cpu())



OutOfMemoryError: CUDA out of memory. Tried to allocate 196.00 MiB. GPU 0 has a total capacity of 47.40 GiB of which 156.44 MiB is free. Process 1663971 has 424.00 MiB memory in use. Process 524715 has 36.96 GiB memory in use. Including non-PyTorch memory, this process has 6.23 GiB memory in use. Of the allocated memory 5.44 GiB is allocated by PyTorch, and 292.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)